# Tutorial on interacting with ReaktoroBlock
Demonstration of how use Reaktoro block in practical settings. 

## Dependencies
* Python - Programming language
* Pyomo - Python package for equation-oriented modeling
* IDAES - Python package extending Pyomo for flowsheet modeling
* cyipopt - Solver necessary for use with gray box models
* Reaktoro-pse - Python package for building Reaktoro gray box models
* WaterTAP - Unit models

## Demonstration structure 
* Background on speciation and Reaktoro-pse philosophy 
* Setting up basic speciation block and calculating properties for a feed composition
    * Demonstrate key configuration options
    * Key display options
    * How to adjust apparent species to achieve thermodynamic zero charge
    * add chemistry 



## Reaktoro-PSE API and structure 
Reaktoro-pse is a wrapper for enabling use of [Reaktoro](https://reaktoro.org/) as a [Graybox model](https://pyomo.readthedocs.io/en/stable/contributed_packages/pynumero/pynumero.interfaces.external_grey_box_model.html) in IDAES compatible modeling platforms such as WaterTAP. This is not a replacement for Reaktoro or higher level API for Reaktoro. 

The general objective is to provide a structure that enables user to automatically build Reaktoro Graybox model by specifying:

* Input ion composition (apprant  or exact species)
* System temperature, pressure, pH, and charge neutrality
* Rekatoro databases and activity models
* Outputs supported by Reaktoro and custom outputs built using Reaktoro database information and outputs 

The general API structure is shown in Figure bellow. The figure shows the type of inputs and outputs and how they are handled by core api to configure Reaktoro Graybox model. 

<img src="reaktoro_pse_api.png" width="1000" height="650">

Reaktoro-pse supports all available databases and activity models provided by Reaktoro:
* Please refer here for information on [databases](https://reaktoro.org/tutorials/basics/loading-databases.html) (all are supported) 
* Please refer here for information on [activity models](https://reaktoro.org/tutorials/basics/specifying-activity-models.html) (all are supported, included chain operations, or passing in pre-configured activity models) 

Reaktoro-pse supports all properties that provide as single floating point or real value as an output from chemical and aqueous properties: 
* [Chemical properties](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html)
* [Aqueous properties](https://reaktoro.org/api/classReaktoro_1_1AqueousProps.html)
* Pyomo build properties, which are custom properties built in Pyomo that use chemical properties or aqueous properties 

## ReaktoroBlock builds Reaktoro Graybox usings standard IDAES StateBlock methods

The ReaktoroBlock automates construction of reaktoro Graybox as a IDAES [StateBlock](https://idaes-pse.readthedocs.io/en/stable/explanations/components/property_package/state_block.html), providing range of configuration of options to simplify performing equilibrium chemistry calculations:

* Enables construction of speciation block and propagation of exact spetiation to property block to enable chemistry modification
* Supports indexing 
* Provides options to configure reaktoro property block (and if constructed speciation block) 
* Uses core api functionality to automatically scale input and output variables and constraints if user does not provide any
* Automatically scales gray box jacobian 
* Provides defaults to simplify configuration and usage for typical calculations 

**Minimal configuration options required for RreaktoroBlock:**
* Database and database file selection from Reaktoro
* Activity models for phases being considered from Reaktoro
* Input apparent or exact species mol or mass flows (**concentrations are not supported!**)
* If speciation block is needed (if exact speciation is not provided then generally yes)
* System states (temperature, pressure, or pH, unless they are being solved for)
* Outputs of interest 

### Example for speciation of NaCl and use with RO model

## Import needed modules

In [120]:
## Import core components
# Pyomo core components
from pyomo.environ import (
    Var,
    Constraint,
    TransformationFactory,
    Reals,
    ConcreteModel,
    value,
    assert_optimal_termination,
    units as pyunits,
)
from pyomo.network import Arc

# Ideas core components
from idaes.core import FlowsheetBlock
from idaes.core.util.scaling import (
    calculate_scaling_factors,
    set_scaling_factor,
    constraint_scaling_transform,
)

from idaes.core.util.model_statistics import degrees_of_freedom
from watertap.core.solvers import get_solver

from idaes.core.util.initialization import propagate_state

from idaes.models.unit_models import Feed
from pyomo.util.calc_var_value import calculate_variable_from_constraint

# WaterTAP core components
import watertap.property_models.NaCl_prop_pack as properties

# Import reaktoro-pse and reaktoro
from reaktoro_pse.reaktoro_block import ReaktoroBlock
import reaktoro

### Incorporating Reaktoro to calculate Seawater properties in Reverse Osmosis Desalination
Sea water desalination is a common process. Here it is demonstrated how ReaktoroBlock can be used:
 1) Calculate feed density, achieve charge neutrality, and estimate amount of acid need to operate at target pH
 2) Calculate osmotic pressure on each node of 1D WaterTAP RO model

In [121]:
"""This a typical composition of sea water with ion concentration in mg/L and pH"""

sea_water_composition = {
    "Na": 10556,
    "K": 380,
    "Ca": 400,
    "Mg": 1262,
    "Cl": 17000,
    "SO4": 2649,
    "HCO3": 140,
}
sea_water_ph = 7.56

### Define standard Pyomo model and WaterTAP property package

In [122]:
m = ConcreteModel()
# create IDAES flowsheet
m.fs = FlowsheetBlock(dynamic=False)
# create NaCl property model
m.fs.properties = properties.NaClParameterBlock()
# build feed
m.fs.feed = Feed(property_package=m.fs.properties)

### Define inputs for Reaktoro and outputs
The NaCl prop package does not track spiceies so we will create new variable sets to track thesese as well as needed outputs

In [123]:
"""Need to create true feed composition, the default NaCl property block only tracks NaCl"""

# Get ions
ions = list(sea_water_composition.keys())
#
m.fs.feed.species_concentrations = Var(
    ions, initialize=1, bounds=(0, None), units=pyunits.mg / pyunits.L
)

"""ReaktoroBlock needs mass flows of all the species"""
ions.append("H2O")
m.fs.feed.species_mass_flow = Var(
    ions, initialize=1, bounds=(0, None), units=pyunits.kg / pyunits.s
)

"""Create pH variable for feed"""
m.fs.feed.pH = Var(initialize=sea_water_ph)
m.fs.feed.pH.fix()

"""The solution is not charge neutralized, so to get true TDS, we will need to adjust concentration of one of the ions"""
m.fs.feed.reaktoro_charge = Var(initialize=0, bounds=(None, None))
set_scaling_factor(m.fs.feed.reaktoro_charge, 1e8)

"""To get true flow mass, the density is needed for the solution, we can get that from reaktoro"""
m.fs.feed.reaktoro_density = Var(
    initialize=1000, units=pyunits.kg / pyunits.m**3
)  # default density unit returned by reaktoro - https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html#afd97c7985258fa29f5a69634c07a0ac7

""" We can also get osmotic pressure to compare to NaCl prop pack predictions, and pH after acid addition"""
m.fs.feed.reaktoro_osmotic_pressure = Var(initialize=1, units=pyunits.Pa)
m.fs.feed.reaktoro_adjusted_pH = Var(initialize=1, units=pyunits.dimensionless)

""" We will want to add a dose of acid to the feed to drop the pH to typical operating point of RO"""
m.fs.acid_dose = Var(initialize=0, units=pyunits.mg / pyunits.L)
m.fs.acid_mass_flow = Var(
    initialize=0, units=pyunits.kg / pyunits.s
)  # Reaktoro will use massflow as input!

### Writing constraints for calculation of mass flows

In [124]:
"""Write constraints to convert concentration to mass flows"""

@m.fs.feed.Constraint(list(m.fs.feed.species_mass_flow.keys()))
def eq_feed_species_mass_flow(fs, ion):
    if ion == "H2O":
        return (
            m.fs.feed.species_mass_flow["H2O"]
            == m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"]
        )
    else:
        """calculate mass flow based on density"""
        return m.fs.feed.species_mass_flow[ion] == pyunits.convert(
            m.fs.feed.species_concentrations[ion]
            * m.fs.feed.properties[0].flow_mass_phase_comp[("Liq", "H2O")]
            / m.fs.feed.reaktoro_density,
            to_units=pyunits.kg / pyunits.s,
        )


""" Need NaCl for the NaCl property package in watertap """
m.fs.feed.eq_NaCl = Constraint(
    expr=m.fs.feed.properties[0].flow_mass_phase_comp[("Liq", "NaCl")]
    == sum(m.fs.feed.species_mass_flow[ion] for ion in m.fs.feed.species_concentrations)
)

""" Calculate acid dose """
m.fs.eq_acid_dose = Constraint(
    expr=m.fs.acid_mass_flow
    == pyunits.convert(
        m.fs.acid_dose
        * m.fs.feed.properties[0].flow_mass_phase_comp[("Liq", "H2O")]
        / m.fs.feed.reaktoro_density,
        to_units=pyunits.kg / pyunits.s,
    )
)

### Reaktoro Block output dict

In [125]:
"""We need to define an output dictionary with our properties - this can also be an Pyomo IndexedVar please check examples in examples folder for how to use IndexedVars as outputs"""

m.fs.feed.reaktoro_outputs = {
    (
        "pH",
        None,
    ): m.fs.feed.reaktoro_adjusted_pH,  # the "None" in second index is defined to indicate its a system property rather then specific component
    (
        "osmoticPressure",
        "H2O",
    ): m.fs.feed.reaktoro_osmotic_pressure,  # not how the second key is the water, we can get osmotic pressure for different components in the system
    ("density", None): m.fs.feed.reaktoro_density,
    ("charge", None): m.fs.feed.reaktoro_charge,
    "speciesAmount": True, } # - this will force reaktor to return exact speciation with all species

### Configure ReaktoroBlock
We are dealing with seawater desalination, which implies operation at high salinity and pressure for such conditions the PhreeqC Pitzer data base is a good choice 
For comparison of PhreeqC data base following paper is a good reference: https://doi.org/10.1016/j.earscirev.2021.103888 

In [126]:
from reaktoro_pse.reaktoro_block_config import reaktoro_solver_options


m.fs.feed.reaktoro_properties = ReaktoroBlock(
    aqueous_phase={
        "composition": m.fs.feed.species_mass_flow,  # This is the spices mass flow
        "convert_to_rkt_species": True,  # We can use default converter as its defined for default database (Phreeqc and pitzer)
        "activity_model": reaktoro.ActivityModelPitzer(),  # Can provide a string, or Reaktoro initialized class
        "fixed_solvent_specie": "H2O",  # We need to define our aqueous solvent as we have to speciate the block
    },
    system_state={
        "temperature": m.fs.feed.properties[0].temperature,
        "pressure": m.fs.feed.properties[0].pressure,
        "pH": m.fs.feed.pH,
    },
    outputs=m.fs.feed.reaktoro_outputs,  # outputs we desired
    chemistry_modifier={
        "HCl": m.fs.acid_mass_flow
    },  # here we define that we are adding 'HCL' as acid and its mass flow
    database="PhreeqcDatabase",  # Can provide a string, or Reaktoro initialized class reaktor.PhreeqcDatabase()
    database_file="pitzer.dat",  # needs to be a string that names the database file or points to its location
    dissolve_species_in_reaktoro=True,  # This will sum up all species into elements in Reaktoro directly, if set to false, it will build Pyomo constraints instead
    assert_charge_neutrality=False,  # This is True by Default, but here we actually want to adjust the input speciation till the charge is zero
    reaktoro_solve_options={
        "open_species_on_property_block": [
            "H+",
            "OH-",
        ]
    },  # This option helps stabilize Reaktoro by providing redundant constraints and generally does not impact final solution.
    build_speciation_block=True,  # We provided apparent species so we need to speciate them.
)

2024-09-02 12:13:36 [INFO] idaes.reaktoro_pse.core.reaktoro_inputs: Exact speciation is not provided! Fixing aqueous solvent and, excluding H
2024-09-02 12:13:36 [INFO] idaes.reaktoro_pse.core.reaktoro_inputs: Exact speciation is not provided! Fixing aqueous solvent and, excluding O
2024-09-02 12:13:36 [INFO] idaes.reaktoro_pse.core.reaktoro_gray_box: RKT gray box using Jt.J hessian type
2024-09-02 12:13:36 [WARNING] idaes.reaktoro_pse.core.reaktoro_inputs: Registered an empty constraint for ['H+', 'OH-'], this can lead to unexpected results depending on reaktoro configuration, please use with caution
2024-09-02 12:13:36 [WARNING] idaes.reaktoro_pse.core.reaktoro_inputs: Registered an empty constraint for ['H+', 'OH-'], this can lead to unexpected results depending on reaktoro configuration, please use with caution
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_gray_box: RKT gray box using Jt.J hessian type


### Inspecting Reaktoro block outputs and inputs

Lets inspect inputs into the Graybox model on speciation block

In [127]:
m.fs.feed.reaktoro_properties.speciation_block.reaktoro_model.inputs.display()

inputs : Size=11, Index=fs.feed.reaktoro_properties.speciation_block.reaktoro_model._input_names_set
    Key         : Lower : Value : Upper : Fixed : Stale : Domain
          CO3-2 :     0 :     1 :  None : False : False :  Reals
           Ca+2 :     0 :     1 :  None : False : False :  Reals
            Cl- :     0 :     1 :  None : False : False :  Reals
            H2O :     0 :     1 :  None : False : False :  Reals
             K+ :     0 :     1 :  None : False : False :  Reals
           Mg+2 :     0 :     1 :  None : False : False :  Reals
            Na+ :     0 :     1 :  None : False : False :  Reals
          SO4-2 :     0 :     1 :  None : False : False :  Reals
             pH :     0 :     1 :  None : False : False :  Reals
       pressure :     0 :     1 :  None : False : False :  Reals
    temperature :     0 :     1 :  None : False : False :  Reals


This inputs are converted to element sums, with in reaktoro and we can inspect them for each block
useing lower level api calls 

In [128]:
""" for speciation block"""

conversion_dict = (
    m.fs.feed.reaktoro_properties.speciation_block.rkt_inputs.constraint_dict
)
for element, species in conversion_dict.items():
    print(element, species)

C [(1.0, 'CO3-2')]
Na [(1.0, 'Na+')]
Mg [(1.0, 'Mg+2')]
S [(1.0, 'SO4-2')]
Cl [(1.0, 'Cl-')]
K [(1.0, 'K+')]
Ca [(1.0, 'Ca+2')]


In [129]:
"""for main block"""

conversion_dict = m.fs.feed.reaktoro_properties.rkt_inputs.constraint_dict
for element, species in conversion_dict.items():
    print(element, species)

H [(2.0, 'H2O'), (1.0, 'H+'), (1.0, 'MgOH+'), (1.0, 'OH-'), (1.0, 'HSO4-'), (1.0, 'HCO3-'), (1, 'HCl')]
C [(1.0, 'MgCO3'), (1.0, 'CO3-2'), (1.0, 'HCO3-'), (1.0, 'CO2')]
O [(1.0, 'H2O'), (3.0, 'MgCO3'), (4.0, 'SO4-2'), (1.0, 'MgOH+'), (3.0, 'CO3-2'), (1.0, 'OH-'), (4.0, 'HSO4-'), (3.0, 'HCO3-'), (2.0, 'CO2')]
Na [(1.0, 'Na+')]
Mg [(1.0, 'MgCO3'), (1.0, 'MgOH+'), (1.0, 'Mg+2')]
S [(1.0, 'SO4-2'), (1.0, 'HSO4-')]
Cl [(1.0, 'Cl-'), (1, 'HCl')]
K [(1.0, 'K+')]
Ca [(1.0, 'Ca+2')]


Lets inspect outputs from speciation block, which produce exact stream speciation based on input

In [130]:
m.fs.feed.reaktoro_properties.speciation_block.outputs.display()

outputs : Size=15, Index={('speciesAmount', 'H2O'), ('speciesAmount', 'MgCO3'), ('speciesAmount', 'H+'), ('speciesAmount', 'K+'), ('speciesAmount', 'SO4-2'), ('speciesAmount', 'MgOH+'), ('speciesAmount', 'Na+'), ('speciesAmount', 'CO3-2'), ('speciesAmount', 'Cl-'), ('speciesAmount', 'Ca+2'), ('speciesAmount', 'OH-'), ('speciesAmount', 'HSO4-'), ('speciesAmount', 'Mg+2'), ('speciesAmount', 'HCO3-'), ('speciesAmount', 'CO2')}
    Key                        : Lower : Value : Upper : Fixed : Stale : Domain
      ('speciesAmount', 'CO2') :  None :     1 :  None : False : False :  Reals
    ('speciesAmount', 'CO3-2') :  None :     1 :  None : False : False :  Reals
     ('speciesAmount', 'Ca+2') :  None :     1 :  None : False : False :  Reals
      ('speciesAmount', 'Cl-') :  None :     1 :  None : False : False :  Reals
       ('speciesAmount', 'H+') :  None :     1 :  None : False : False :  Reals
      ('speciesAmount', 'H2O') :  None :     1 :  None : False : False :  Reals
    ('specie

Lets inspect inputs into main ReaktoroBlock Graybox

Note how the speciation block outputs are now inputs into the model, including chemistry modifier HCl!

In [131]:
m.fs.feed.reaktoro_properties.reaktoro_model.inputs.display()

inputs : Size=18, Index=fs.feed.reaktoro_properties.reaktoro_model._input_names_set
    Key         : Lower : Value : Upper : Fixed : Stale : Domain
            CO2 :     0 :     1 :  None : False : False :  Reals
          CO3-2 :     0 :     1 :  None : False : False :  Reals
           Ca+2 :     0 :     1 :  None : False : False :  Reals
            Cl- :     0 :     1 :  None : False : False :  Reals
             H+ :     0 :     1 :  None : False : False :  Reals
            H2O :     0 :     1 :  None : False : False :  Reals
          HCO3- :     0 :     1 :  None : False : False :  Reals
            HCl :     0 :     1 :  None : False : False :  Reals
          HSO4- :     0 :     1 :  None : False : False :  Reals
             K+ :     0 :     1 :  None : False : False :  Reals
           Mg+2 :     0 :     1 :  None : False : False :  Reals
          MgCO3 :     0 :     1 :  None : False : False :  Reals
          MgOH+ :     0 :     1 :  None : False : False :  Reals
      

Lets inspect outputs from main reaktoro model

Note how we are missing osmoticPressure, and instead have speciesActivityLn and speciesStandardVolume as our outputs, this is becouse we osmoticPressure is a pyomo property, rather a native property supplied by reaktoro. You can inspect how this property is created by checking the:
* osmoticPressure in PyomoProperties class located in reaktoro_pse.core.reaktoro_outputs
* build_osmotic_constraint in reaktoro_pse.core.pyomo_property_writer.property_functions

This two location will also show how we can access reaktoro database to pull out fixed parameters and create a custom pyomo property. 

In [132]:
m.fs.feed.reaktoro_properties.reaktoro_model.outputs.display()

outputs : Size=21, Index=fs.feed.reaktoro_properties.reaktoro_model._output_names_set
    Key                              : Lower : Value : Upper : Fixed : Stale : Domain
                    ('charge', None) :  None :   0.1 :  None : False : False :  Reals
                   ('density', None) :  None :   0.1 :  None : False : False :  Reals
                        ('pH', None) :  None :   0.1 :  None : False : False :  Reals
        ('speciesActivityLn', 'H2O') :  None :   0.1 :  None : False : False :  Reals
            ('speciesAmount', 'CO2') :  None :   0.1 :  None : False : False :  Reals
          ('speciesAmount', 'CO3-2') :  None :   0.1 :  None : False : False :  Reals
           ('speciesAmount', 'Ca+2') :  None :   0.1 :  None : False : False :  Reals
            ('speciesAmount', 'Cl-') :  None :   0.1 :  None : False : False :  Reals
             ('speciesAmount', 'H+') :  None :   0.1 :  None : False : False :  Reals
            ('speciesAmount', 'H2O') :  None :   0.1 :

Now lets check the final reaktoro output that are built on ReaktoroBlock

Note how only specie amounts are present in an IndexedVar other properties are automatically connected to Variables we provided in our m.fs.feed.reaktoro_outputs dictionary. 

In [133]:
m.fs.feed.reaktoro_properties.outputs.display()

outputs : Size=15, Index={('speciesAmount', 'H2O'), ('speciesAmount', 'MgCO3'), ('speciesAmount', 'H+'), ('speciesAmount', 'K+'), ('speciesAmount', 'SO4-2'), ('speciesAmount', 'MgOH+'), ('speciesAmount', 'Na+'), ('speciesAmount', 'CO3-2'), ('speciesAmount', 'Cl-'), ('speciesAmount', 'Ca+2'), ('speciesAmount', 'OH-'), ('speciesAmount', 'HSO4-'), ('speciesAmount', 'Mg+2'), ('speciesAmount', 'HCO3-'), ('speciesAmount', 'CO2')}
    Key                        : Lower : Value : Upper : Fixed : Stale : Domain
      ('speciesAmount', 'CO2') :  None :     1 :  None : False : False :  Reals
    ('speciesAmount', 'CO3-2') :  None :     1 :  None : False : False :  Reals
     ('speciesAmount', 'Ca+2') :  None :     1 :  None : False : False :  Reals
      ('speciesAmount', 'Cl-') :  None :     1 :  None : False : False :  Reals
       ('speciesAmount', 'H+') :  None :     1 :  None : False : False :  Reals
      ('speciesAmount', 'H2O') :  None :     1 :  None : False : False :  Reals
    ('specie

### Set default values for feed and scaling

In [134]:
m.fs.feed.properties[0].temperature.fix(273 + 25)  # temperature (K)
m.fs.feed.properties[0].pressure.fix(101325)  # pressure (Pa)
m.fs.feed.properties[0].flow_mass_phase_comp["Liq", "H2O"].fix(
    0.965
)  # mass flowrate of H2O (kg/s)
m.fs.feed.properties[0].conc_mass_phase_comp[...]  # construct concentration props
m.fs.feed.properties[0].pressure_osm_phase[...]
m.fs.properties.set_default_scaling(
    "flow_mass_phase_comp",
    1 / 0.965,
    index=("Liq", "H2O"),
)
m.fs.properties.set_default_scaling(
    "flow_mass_phase_comp",
    1 / 0.035,  # aproximage scale
    index=("Liq", "NaCl"),
)

### Initialize our composition constraints and scale all the variables

In [135]:
for ion, value in sea_water_composition.items():
    m.fs.feed.species_concentrations[ion].fix(value)
    set_scaling_factor(m.fs.feed.species_concentrations[ion], 1 / value)

for comp, pyoobj in m.fs.feed.eq_feed_species_mass_flow.items():
    calculate_variable_from_constraint(m.fs.feed.species_mass_flow[comp], pyoobj)
    set_scaling_factor(
        m.fs.feed.species_mass_flow[ion], 1 / m.fs.feed.species_mass_flow[comp].value
    )
    constraint_scaling_transform(pyoobj, 1 / m.fs.feed.species_mass_flow[comp].value)


calculate_variable_from_constraint(
    m.fs.feed.properties[0].flow_mass_phase_comp[("Liq", "NaCl")], m.fs.feed.eq_NaCl
)
constraint_scaling_transform(m.fs.feed.eq_NaCl, 1 / 0.035)

set_scaling_factor(m.fs.feed.reaktoro_density, 1 / 1000)
set_scaling_factor(m.fs.feed.reaktoro_osmotic_pressure, 1 / 1e5)
set_scaling_factor(m.fs.feed.reaktoro_adjusted_pH, 1)
set_scaling_factor(m.fs.feed.pH, 1)
set_scaling_factor(m.fs.acid_mass_flow, 1e2)
set_scaling_factor(m.fs.acid_dose, 1e-3)
set_scaling_factor(m.fs.feed.reaktoro_charge, 1e8)

### Intialize feed and reaktoro block. 
Reaktoro initialization does several steps:

1) Initialize input constraints propagating them from user variables to Reaktoro graybox inputs
2) Solve the Reaktoro block to get output properties 
3) Propagate Reaktoro solution through output constraints and to output variables 
4) Scale all input and output variables and constraints using either user provided scaling factors or by inverse of their value 
5) Scale the jacobian using user provided scaling or inverse of scaling factors of the gray box outputs

This will in general provide a well scaled problem. 

In [136]:
solver = get_solver()  # get solver
m.fs.feed.initialize(optarg=solver.options)
m.fs.feed.reaktoro_properties.initialize()
""" this is our exact composition!"""
# m.fs.feed.reaktoro_properties.outputs.display()

""" and these are manually defined properties and variables of interest """
print(
    "Density reaktoro",
    m.fs.feed.reaktoro_density.value,
    "Density NaCl prop pack",
    m.fs.feed.properties[0].dens_mass_phase["Liq"].value,
)
print(
    "Osmotic pressure",
    m.fs.feed.reaktoro_osmotic_pressure.value,
    "Density NaCl prop pack",
    m.fs.feed.properties[0].pressure_osm_phase["Liq"].value,
)
print("Adjusted pH", m.fs.feed.reaktoro_adjusted_pH.value)
print("Solution reaktoro_charge", m.fs.feed.reaktoro_charge.value)

2024-09-02 12:13:37 [INFO] idaes.init.fs.feed: Initialization Complete.
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_state: Equilibrated successfully
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Initialized rkt block
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_state: Equilibrated successfully
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Initialized rkt block
Density reaktoro 1022.0762152208728 Density NaCl prop pack 1018.7164667900701
Osmotic pressure 2306930.392393324 Density NaCl prop pack 2542018.9084112146
Adjusted pH 7.560000000075305
Solution reaktoro_charge 0.05380713824779206


D:\github\reaktoro-pse\src\reaktoro_pse\core\reaktoro_jacobian.py:323: RuntimeWarning: invalid value encountered in divide
  np.diff(values) / np.diff(self.numerical_steps * input_value)


We can check the jacobian scaling using provided tool, it will also return a dictionary with scaling factors. 

In [137]:
jac_scaling_dict = m.fs.feed.reaktoro_properties.display_jacobian_scaling()

2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.reaktoro_block: -----Displaying information for speciation block ------
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Jacobian scale for ('speciesAmount', 'H+') : 3.6980404144745577e-08, IDX: 0
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Jacobian scale for ('speciesAmount', 'H2O') : 53.563499111900526, IDX: 1
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Jacobian scale for ('speciesAmount', 'CO3-2') : 2.1445433578637287e-05, IDX: 2
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Jacobian scale for ('speciesAmount', 'CO2') : 8.114686961714109e-05, IDX: 3
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Jacobian scale for ('speciesAmount', 'Ca+2') : 0.009631002164385654, IDX: 4
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_block_builder: Jacobian scale for ('speciesAmount', 'Cl-') : 0.4627

In [138]:
m.fs.feed.reaktoro_properties.display_jacobian_outputs()

2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.reaktoro_block: -----Displaying information for speciation block ------
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_jacobian: -----displaying jacobian outputs and types-----
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_jacobian: ('speciesAmount', 'H+'): Jac type: exact
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_jacobian: ('speciesAmount', 'H2O'): Jac type: exact
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_jacobian: ('speciesAmount', 'CO3-2'): Jac type: exact
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_jacobian: ('speciesAmount', 'CO2'): Jac type: exact
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_jacobian: ('speciesAmount', 'Ca+2'): Jac type: exact
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_jacobian: ('speciesAmount', 'Cl-'): Jac type: exact
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.core.reaktoro_jacobian: ('speciesAmount', 'H

## Updating Jacobian scaling 
In some cases we need to manually scale output Jacobian values, as they can get overscaled via automatic scaling.
This can be done during creating of ReaktoroBlock by passing "jacobian_user_scaling" option with a dictionary of output indexes and scaling factor to apply. In some cases it might be more convenient to update scaling factor later.

Example below shows updating scaling factor using built in function set_jacboian_scaling, the scaling will be applied to both speciation block (if built) and main block


In [139]:
m.fs.feed.reaktoro_properties.set_jacobian_scaling({("density", None): 1000})

In [140]:
m.fs.feed.reaktoro_properties.display_reaktoro_state()

2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.reaktoro_block: -----Displaying information for speciation block ------
2024-09-02 12:13:37 [INFO] idaes.reaktoro_pse.reaktoro_block: +-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |   298.0000 |    K |
| Pressure        |     1.0132 |  bar |
| Charge:         | 5.3807e-02 |  mol |
| Element Amount: |            |      |
| :: H            | 1.0713e+02 |  mol |
| :: C            | 2.2512e-03 |  mol |
| :: O            | 5.3677e+01 |  mol |
| :: Na           | 4.4310e-01 |  mol |
| :: Mg           | 5.0108e-02 |  mol |
| :: S            | 2.6610e-02 |  mol |
| :: Cl           | 4.6272e-01 |  mol |
| :: K            | 9.3791e-03 |  mol |
| :: Ca           | 9.6310e-03 |  mol |
| Species Amount: |            |      |
| :: H+           | 3.6980e-08 |  mol |
| :: H2O          | 5.3563e+01 |  mol |
| :: CO3-2        | 2.1445e-05 |  mol |
| :: CO2          | 

### Lets solve the current model to:
* Find actual mass flows of species 
* Solution density
* Required Cl amount to get zero charge in solution
* Dose of HCl ot get to pH of 7

In [141]:
""" unfix Cl and fix charge to 0"""

m.fs.feed.species_concentrations["Cl"].unfix()
m.fs.feed.reaktoro_charge.fix(0)
""" fix our target pH"""
m.fs.feed.reaktoro_adjusted_pH.fix(7)

Lets check DOFs before solve, and note that its equal to number of our reaktoro outputs

In [142]:
print("DOFs:", degrees_of_freedom(m))
outputs_main_block = len(m.fs.feed.reaktoro_properties.reaktoro_model.outputs)
outputs_speciation_block = len(
    m.fs.feed.reaktoro_properties.speciation_block.reaktoro_model.outputs
)
print("Number of Reaktoro speciation block outputs", outputs_speciation_block)
print("Number of Reaktoro outputs", outputs_main_block)
print(
    "Total number of outputs on reaktoro", outputs_main_block + outputs_speciation_block
)
print(
    "Actual DOFs:",
    degrees_of_freedom(m) - (outputs_main_block + outputs_speciation_block),
)
assert degrees_of_freedom(m) - (outputs_main_block + outputs_speciation_block) == 0

DOFs: 36
Number of Reaktoro speciation block outputs 15
Number of Reaktoro outputs 21
Total number of outputs on reaktoro 36
Actual DOFs: 0


In [143]:
initial_cl = m.fs.feed.species_concentrations["Cl"].value
cy_solver = get_solver(solver="cyipopt-watertap")

cy_solver.options["max_iter"] = 25

result = cy_solver.solve(m, tee=True)
assert_optimal_termination(result)

cyipopt-watertap: cyipopt with user variable scaling and IDAES jacobian constraint scaling
This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.2.

Number of nonzeros in equality constraint Jacobian...:      746
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       12

Total number of variables............................:      117
                     variables with only lower bounds:       41
                variables with lower and upper bounds:        6
                     variables with only upper bounds:        0
Total number of equality constraints.................:      117
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du

In [144]:
print(
    "Density reaktoro",
    m.fs.feed.reaktoro_density.value,
    "Density NaCl prop pack",
    m.fs.feed.properties[0].dens_mass_phase["Liq"].value,
)
print(
    "Osmotic pressure",
    m.fs.feed.reaktoro_osmotic_pressure.value,
    "Density NaCl prop pack",
    m.fs.feed.properties[0].pressure_osm_phase["Liq"].value,
)
print(
    "Adjusted pH",
    m.fs.feed.reaktoro_adjusted_pH.value,
    "Acid dose",
    m.fs.acid_dose.value,
    "Acid mass flow",
    m.fs.acid_mass_flow.value,
)
print(
    "Solution reaktoro_charge",
    m.fs.feed.reaktoro_charge.value,
    "intial Cl",
    initial_cl,
    "final Cl",
    m.fs.feed.species_concentrations["Cl"].value,
)

Density reaktoro 1022.4474912371852 Density NaCl prop pack 1019.5824469406042
Osmotic pressure 2386602.5446863747 Density NaCl prop pack 2639273.7868881626
Adjusted pH 7 Acid dose 8.43304628245746 Acid mass flow 7.959225028489641e-06
Solution reaktoro_charge 0 intial Cl 17000 final Cl 18976.76540015965
